In [1]:
import pandas as pd #version 1.4.4
import numpy as np #version 1.21.5
import matplotlib.pyplot as plt
import sklearn

In [2]:
#this function reads in the data set

dance_injuries_2023= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2023.csv')
dance_injuries_2022= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2022.csv')
dance_injuries_2021= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2021.csv')
dance_injuries_2020= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2020.csv')
dance_injuries_2019= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2019.csv')
dance_injuries_2018= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2018.csv')
dance_injuries_2017= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2017.csv')
dance_injuries_2016= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2016.csv')
dance_injuries_2015= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2015.csv')
dance_injuries_2014= pd.read_csv('/Users/christinegurek/Desktop/Data 400 Docs/Mini-Project/Dance Mini Project Research/csv/Injuries_2014.csv')

injuries_total = pd.concat([dance_injuries_2014,dance_injuries_2015,dance_injuries_2016,dance_injuries_2017,dance_injuries_2018,dance_injuries_2019,dance_injuries_2020,dance_injuries_2021,dance_injuries_2022,dance_injuries_2023], axis=0)
injuries_total = injuries_total.drop(columns=['Product_1', 'Product_2', 'Product_3'])
injuries_total


,CPSC_Case_Number,Treatment_Date,Age,Body_Part,Diagnosis,Narrative
0,140122872,1/6/14,18,83,64,"18 YO F, C/O LT FOOT PAIN AFTER PERFORMING MOD..."
1,140130758,1/9/14,12,36,64,12YR F DANCING IN DANCE RECITAL;DX LOW LEG STRAIN
2,140136188,1/15/14,8,79,64,8YF TAKING DANCING CLASS NOW WITH UPPER ABD PA...
3,140137784,1/16/14,13,89,64,13YOF AT A SCHOOL DANCE AND WHILE DANCING FELT...
4,140141186,1/15/14,13,79,64,13 YOF PARTICIPATING IN DANCE CLASS STRAINED R...
...,...,...,...,...,...,...
494,240123734,12/4/23,8,35,55,8YOF WAS IN DANCE CLASS AND WAS PERFORMING A D...
495,240158321,12/4/23,12,79,71,12YOF WITH LOWER BACK AND FLANK PAIN FOR 1 WEE...
496,240162316,12/4/23,14,35,64,14YOF INJURING HER LEFT KNEE DURING DANCE CLAS...
497,240166190,12/12/23,11,35,71,11YOF FELL ON THE FLOOR DURING DANCE CLASS. DX...


In [3]:
# #text analysis
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
import nltk

# Load the data
narratives = injuries_total['Narrative'].dropna()  # Drop any missing values in 'Narrative' column

# Preprocess text data
nltk.download('stopwords')
stop_words = stopwords.words('english')
vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words=stop_words)
narrative_vectors = vectorizer.fit_transform(narratives)

# Train the LDA model
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)  # Adjust n_components for more/fewer topics
lda_model.fit(narrative_vectors)

# Extract and display topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}: ", " ".join([feature_names[i] for i in topic.argsort()[-no_top_words:]]))

no_top_words = 10  # Number of words per topic to display
feature_names = vectorizer.get_feature_names_out()
display_topics(lda_model, feature_names, no_top_words)

# df = injuries_total
# df

# import gensim
# from gensim import corpora
# from gensim.models import LdaModel

# def preprocess_text(text):
#     return text.lower().split()

# df['processed_text']= df['Narrative'].apply(preprocess_text)
# df

# dictionary = corpora.Dictionary(df['processed_text'])
# corpus = [dictionary.doc2bow(text) for text in df['processed_text']]

# num_topics = 3 
# lda_analysis = LdaModel(corpus, num_topics = num_topics, id2word = dictionary, passes = 15)


# for topic in lda_analysis.print_topics():
#     print(topic)

    
# doc = df['processed_text'][0]
# doc_bow = dictionary.doc2bow(doc)
# topic_distribution = lda_analysis[doc_bow]
# print(topic_distribution)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christinegurek/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Topic 1:  fell yof strain neck shoulder class dx back pain dance
Topic 2:  practice yo hurt twisted yof class sprain dx dance ankle
Topic 3:  sprain elbow left fell yof pain class dx dance foot
Topic 4:  hit injury chest hip head class fell pain dx dance
Topic 5:  dislocation right class pop felt left pain dx dance knee


In [ ]:
import pandas as pd #version 1.4.4
import sklearn as skl #version 1.0.2
import numpy as np # version 1.21.5
import warnings as ww

from sklearn.utils import resample #version 1.0.2
from sklearn.model_selection import train_test_split #version 1.0.2
from sklearn.ensemble import RandomForestClassifier #version 1.0.2
from sklearn.tree import export_text #version 1.0.2
from sklearn.svm import SVC #version 1.0.2

from sklearn.metrics import confusion_matrix #version 1.0.2
from sklearn.metrics import classification_report #version 1.0.2

ww.filterwarnings("ignore")

# Convert Treatment_Date to datetime format, then extract year and month as features
injuries_total['Treatment_Date'] = pd.to_datetime(injuries_total['Treatment_Date'], errors='coerce')
injuries_total['Treatment_Year'] = injuries_total['Treatment_Date'].dt.year
injuries_total['Treatment_Month'] = injuries_total['Treatment_Date'].dt.month

# Drop the original Treatment_Date as we now have numerical year and month columns
injuries_total = injuries_total.drop(columns=['Treatment_Date', 'Narrative']) 


#classification models

# this splits the data into training and test sets

y = injuries_total["Diagnosis"]
x = injuries_total.loc[:, injuries_total.columns != "Diagnosis"] 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

#Random Forest Classifier
RF_model1 = RandomForestClassifier()
RF_model1.fit(X_train, y_train)
y_pred1 = RF_model1.predict(X_test)
y_pred1

# this runs a confusion matrix and classification report so we can see 
# different error metrics of the model

print(confusion_matrix(y_test, y_pred1))
print(classification_report(y_test, y_pred1))

#this displays the decision rules of the random forest classifier

decision = RF_model1.estimators_[0]
all_decisions = export_text(decision, feature_names=list(x.columns))
print(all_decisions)

# this gives us the names of the top 5 most important features on the
# model
RF_model1.feature_importances_
# print((injuries_total.columns[89]))
# print((injuries_total.columns[39]))
# print((injuries_total.columns[29]))
# print((injuries_total.columns[18]))
# print((injuries_total.columns[15]))

#SVM MOdel
SVM_model1 = SVC()
SVM_model1.fit(X_train, y_train)
y_pred_SVM1 = SVM_model1.predict(X_test)
y_pred_SVM1

# this runs a confusion matrix and classification report so we can see 
# different error metrics of the model

print(confusion_matrix(y_test, y_pred_SVM1))
print(classification_report(y_test, y_pred_SVM1, zero_division = 0.0))


#SVM Model 2
SVM_model2 = SVC(kernel = 'linear')
SVM_model2.fit(X_train, y_train)
y_pred_SVM2 = SVM_model2.predict(X_test)
y_pred_SVM2

print(confusion_matrix(y_test, y_pred_SVM2))
print(classification_report(y_test, y_pred_SVM2))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0]
 [  0   0  13   0   0   0   0   0   0   1   0  17   0   0   0   2   0   0]
 [  0   0   1  16   0   3   1   6   0   5   1   1  32   0   0  20   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   0   0   1   0   6   0   1   0   0   0   0  17   0   0  20   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0]
 [  0   0   0  12   0   6   0  30   0   2   0   0  48   0   0  25   0   0]
 [  0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   1   0]
 [  0   0   1   2   0   0   0   2   0  14   1   0   4   0   0   3   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   0  11   2   0   0   0   0   0   0   0  18   0   0   0   1   0   0]
 [  0   0   0  18   0   8   0  20   0   1   0   0 215   0   0  91   0   0]
 [  0   0   0   0   0   0

[[  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  33   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  86   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  45   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 123   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  27   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  32   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 353   0   0   0   0   0]
 [  0   0   0   0   0   0